In [1]:
import ast
import re
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

In [2]:
df = pd.read_csv('povarenok_recipes_2021_06_16.csv')
df.head()

,url,name,ingredients
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146582 entries, 0 to 146581
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   url          146582 non-null  object
 1   name         146582 non-null  object
 2   ingredients  146581 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [5]:
type(df["ingredients"][0])

str

In [6]:
# Функция для преобразования строки в dict
def to_dict(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else {}
    except:
        return {}

# Превращаем колонку в словари
df['ingredients'] = df['ingredients'].apply(to_dict)

In [7]:
def clean_ingredient(ing):
  result = {}
  if isinstance(ing, dict):
    for k, v in ing.items():
      k = k.strip().lower()
      v = str(v).strip().lower()

      match = re.match(r'([\d.,]+)?\s*([а-яa-z]*)', v) # проблема начинается где-то тут, я проверил на другом экземпляре и выше код делает все правильно и как должно быть, а тут начинается проблема
      qty = match.group(1)
      unit = match.group(2)
      result[k] = {'quantity': qty, 'unit': unit}
  return result

df['ingredients'] = df['ingredients'].apply(clean_ingredient)
df.head()

,url,name,ingredients
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'молоко': {'quantity': '250', 'unit': 'мл'}, ..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'сыр твердый': {'quantity': None, 'unit': 'no..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'баклажан': {'quantity': '3', 'unit': 'шт'}, ..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'фарш куриный': {'quantity': '800', 'unit': '..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'вишня': {'quantity': '1', 'unit': 'кг'}, 'во..."


In [8]:
df['ingredients'].iloc[0]

{'молоко': {'quantity': '250', 'unit': 'мл'},
 'клубника': {'quantity': '200', 'unit': 'г'},
 'сахар': {'quantity': '15', 'unit': 'г'}}

In [9]:
ingredient_values = defaultdict(list)
ingredient_units = defaultdict(list)

for ing_dict in df['ingredients']:
  for name, info in ing_dict.items():
    qty = info.get('quantity')
    unit = info.get('unit')

    if qty not in [None, 'none', np.nan, 'None']:
      try:
        qty_val = float(str(qty).replace(',','.'))
        ingredient_values[name].append(qty_val)
      except:
        pass
    if unit not in [None, 'none', np.nan, 'None']:
      ingredient_units[name].append(unit)

ingredient_means = {k: np.mean(v) for k, v in ingredient_values.items() if v}
ingredient_mode_unit = {k: Counter(v).most_common(1)[0][0] for k, v in ingredient_units.items() if v}

In [10]:
def fill_nan_quantities_and_units(ing_dict):
  new_dict = {}
  for name, info in ing_dict.items():
    qty = info.get('quantity')
    unit = info.get('unit')

    if qty not in [None, 'none', np.nan, 'None']:
      new_qty = round(ingredient_means.get(name, 1.0), 2)
    else:
      try:
        new_qty = float(str(qty).replace(',','.'))
      except:
        new_qty = 1.0

    if unit not in [None, 'none', np.nan, 'None']:
      new_unit = unit
    else:
      new_unit = ingredient_mode_unit.get(name, 'шт')

    new_dict[name] = {'quantity': new_qty, 'unit': new_unit}
  return new_dict

df['ingredients'] = df['ingredients'].apply(fill_nan_quantities_and_units)

In [11]:
# Пример: случайно выбрать 14 блюд на неделю (2 в день)
week_meals = df.sample(14, random_state=42)

In [12]:
def aggregate_ingredients(selected_meals):
    shopping_list = defaultdict(lambda: {'quantity': 0.0, 'unit': None})

    for ing_dict in selected_meals['ingredients']:
        for name, info in ing_dict.items():
            qty = info.get('quantity')
            unit = info.get('unit')

            # Преобразуем количество в число (если возможно)
            try:
                qty_val = float(str(qty).replace(',', '.')) if qty else 1.0
            except:
                qty_val = 1.0

            # Если единицы измерения совпадают — суммируем
            if shopping_list[name]['unit'] in [unit, None]:
                # Если quantity вдруг оказался строкой, сбрасываем в 0
                if not isinstance(shopping_list[name]['quantity'], (int, float)):
                    shopping_list[name]['quantity'] = 0.0

                shopping_list[name]['quantity'] += qty_val
                shopping_list[name]['unit'] = unit
            else:
                # Разные единицы — добавляем как текстовое примечание
                existing_qty = str(shopping_list[name]['quantity'])
                shopping_list[name]['quantity'] = f"{existing_qty} + {qty} {unit}"

    return shopping_list

In [13]:
def show_recipes(df, dish_names):
    """
    Выводит названия блюд и их ингредиенты для списка блюд из DataFrame.
    """
    for dish in dish_names:
        row = df.loc[df['name'] == dish]

        if row.empty:
            print(f"Блюдо '{dish}' не найдено.\n")
            continue

        ingredients = row.iloc[0]['ingredients']

        # если ингредиенты записаны как строка — конвертируем
        if isinstance(ingredients, str):
            try:
                ingredients = ast.literal_eval(ingredients)
            except Exception as e:
                print(f"⚠️ Не удалось распарсить ингредиенты для '{dish}': {e}")
                continue

        print(f"{dish}\nИнгредиенты:")
        for name, info in ingredients.items():
            quantity = info.get('quantity', '')
            unit = info.get('unit', '')
            print(f" - {name}: {quantity} {unit}")
        print("\n" + "-" * 40 + "\n")

In [15]:
print(f"Meals for a week\n")

shopping_list = aggregate_ingredients(week_meals)
dish_name_list = [k for k in week_meals['name']]
show_recipes(week_meals, dish_name_list)




Meals for a week

Итaльянский розмариновый кекс
Ингредиенты:
 - яйцо куриное: 2.86 шт
 - сахар: 61.59 
 - масло оливковое: 14.85 
 - розмарин: 1.77 
 - мука пшеничная: 141.88 
 - порошок пекарский: 2.2 ст
 - соль: 1.0 по

----------------------------------------

Японский фруктовый мусс
Ингредиенты:
 - яблоко: 40.18 шт
 - груша: 39.42 шт
 - киви: 13.63 шт
 - апельсин: 8.74 шт
 - белок яичный: 11.14 шт

----------------------------------------

Хлебные котлетки к жареной рыбе
Ингредиенты:
 - хлеб: 43.95 
 - молоко: 139.73 
 - фета: 124.91 
 - сыр твердый: 116.37 
 - яйцо куриное: 2.86 шт
 - сухари панировочные: 1.0 ст
 - лук зеленый: 1.0 пуч
 - петрушка: 1.0 пуч
 - специи: 1.0 по
 - йогурт: 144.31 бан
 - лимон: 2.27 
 - масло оливковое: 1.0 ст
 - чеснок: 4.74 
 - рыба: 203.56 
 - горчица: 3.43 ст
 - мука пшеничная: 141.88 стак

----------------------------------------

Хумус из зеленой гречки
Ингредиенты:
 - крупа гречневая: 56.92 стак
 - петрушка: 6.05 
 - тархун: 16.88 
 - оливки зеле

In [18]:
print(f"\n\nIngredients\n")

n = 1
for k, v in shopping_list.items():
    print(f"{n}. {k}: {v['quantity']} {v['unit'] or ''}")
    n += 1



Ingredients

1. яйцо куриное: 21.02 шт
2. сахар: 61.59 + 61.59 ст + 61.59 стак + 61.59 ст 
3. масло оливковое: 14.85 + 1.0 ст + 14.85 ст 
4. розмарин: 1.77 
5. мука пшеничная: 141.88 + 141.88 стак + 141.88 стак + 141.88 ст + 141.88 г + 141.88 г 
6. порошок пекарский: 2.2 ст
7. соль: 3.0 по
8. яблоко: 40.18 шт
9. груша: 39.42 шт
10. киви: 13.63 шт
11. апельсин: 17.48 шт
12. белок яичный: 11.14 шт
13. хлеб: 43.95 + 1.0 ломт 
14. молоко: 139.73 + 139.73 стак + 139.73 мл + 139.73 стак 
15. фета: 124.91 
16. сыр твердый: 116.37 + 1.0 г + 116.37 г 
17. сухари панировочные: 1.0 ст
18. лук зеленый: 1.0 пуч
19. петрушка: 1.0 пуч
20. специи: 2.0 + 2.06 ч по
21. йогурт: 144.31 бан
22. лимон: 2.27 
23. чеснок: 4.74 + 4.74 зуб + 4.74 зуб 
24. рыба: 203.56 
25. горчица: 3.43 + 3.43 ч ст
26. крупа гречневая: 56.92 стак
27. тархун: 16.88 
28. оливки зеленые: 32.58 шт
29. мука кунжутная: 7.3 ст
30. соевый соус: 23.08 ст
31. сок лимонный: 4.73 ст
32. свекла: 54.15 шт
33. огурец маринованный: 23.85 шт


In [21]:
def test(df):
    week_meals = df.sample(14)
    shopping_list = aggregate_ingredients(week_meals)
    print(f"Meals for a week\n")

    shopping_list = aggregate_ingredients(week_meals)
    dish_name_list = [k for k in week_meals['name']]
    show_recipes(week_meals, dish_name_list)
    
    print(f"\n\nIngredients\n")

    n = 1
    for k, v in shopping_list.items():
        print(f"{n}. {k}: {v['quantity']} {v['unit'] or ''}")
        n += 1

test(df)

Meals for a week

Куриное филе с сыром дорблю и беконом
Ингредиенты:
 - бедро куриное: 104.14 шт
 - сыр голубой: 84.3 г
 - соевый соус: 11.54 ст
 - соус: 1.0 ст
 - чеснок: 4.74 зуб
 - овощи: 1.0 г
 - зелень: 1.0 пуч
 - бекон: 1.0 г
 - соль: 1.0 по

----------------------------------------

Соленые огурцы с красной смородиной на зиму
Ингредиенты:
 - огурец: 21.9 г
 - смородина красная: 126.51 г
 - перец черный: 2.57 шт
 - чеснок: 4.74 зуб
 - укроп: 5.66 шт
 - соль: 4.11 ст
 - сахар: 61.59 ч

----------------------------------------

Щи рахмановские
Ингредиенты:
 - рыба: 1.0 г
 - щавель: 1.0 пуч
 - лук репчатый: 13.66 шт
 - морковь: 29.85 шт
 - картофель: 82.89 шт
 - вода: 129.49 л
 - сметана: 1.0 г
 - соль: 1.0 по
 - перец душистый: 1.0 шт
 - лист лавровый: 1.0 шт

----------------------------------------

Главная  > 
  Рецепты
        > 
    
версия для печати
Ингредиенты:
 - мука пшеничная: 141.88 г
 - масло сливочное: 92.05 г
 - сахар: 61.59 г
 - яйцо куриное: 2.86 шт
 - соль: 4.11 щ